In [1]:
import os
import glob

from pathlib import Path
import shutil

import tensorflow as tf
import tf_keras as keras
import numpy as np
from tqdm import tqdm
import yaml

from rl.network import ResNet
from rl.mcts import MCTS
from rl.buffer import ReplayBuffer, Sample
from rl.game import Game, encode_state

with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
base_path = "graphs"
index = "20241201"
qubits = config["game_settings"]["N"]
training_settings = config["training_settings"]
network_settings = config["network_settings"]
mcts_settings = config["mcts_settings"]
num_cpus = training_settings["num_cpus"]
num_gpus = training_settings["num_gpus"]
n_episodes = training_settings["n_episodes"]
buffer_size = training_settings["buffer_size"]
batch_size = training_settings["batch_size"]
epochs_per_update = training_settings["epochs_per_update"]
update_period = training_settings["update_period"]
save_period = training_settings["save_period"]
eval_period = training_settings.get("eval_period", 100)


def selfplay(weights, qubits, current_episode, config):
    record = []
    game = Game(qubits, config)
    state = game.get_initial_state()
    game.reset_used_columns()
    network = ResNet(action_space=game.action_space, config=config)
    network.predict(encode_state(state, qubits))
    network.set_weights(weights)

    mcts = MCTS(qubits=qubits, network=network, config=config)
    done = False
    total_score = 0
    step_count = 0
    prev_action = None

    while not done and step_count < game.MAX_STEPS:
        mcts_policy = mcts.search(
            root_state=state,
            prev_action=prev_action,
            num_simulations=mcts_settings["num_mcts_simulations"],
        )
        if prev_action is not None:
            indices = [i for i in range(game.action_space) if i != prev_action]
            valid_actions = game.get_valid_actions(state, prev_action)
            prob = mcts_policy[valid_actions]
            prob = prob / prob.sum()
            action = np.random.choice(valid_actions, p=prob)
        else:
            indices = list(range(game.action_space))
            prob = mcts_policy
            action = np.random.choice(indices, p=prob)
        record.append(Sample(state.copy(), mcts_policy, reward=None))
        state, done, action_score = game.step(state, action, prev_action)
        prev_action = action
        # print(state, action_score, done)
        total_score += action_score
        step_count += 1

    reward = game.get_reward(state, total_score)
    for sample in record:
        sample.reward = reward
    return record


def evaluate_self_play(qubits, network, config):
    pattern = os.path.join(base_path, f"adj_matrix_{qubits}_*.npy")
    file_paths = glob.glob(pattern)
    avg_depth = []
    avg_counts = []
    for file_path in file_paths:
        state = np.load(file_path)
        game = Game(qubits, config)
        swap_pairs = []
        done = False
        step_count = 0
        prev_action = None
        while not done and step_count < game.MAX_STEPS:
            encoded_state = encode_state(state, qubits)
            input_state = np.expand_dims(encoded_state, axis=0)
            policy_output, value_output = network.predict(input_state, verbose=0)
            policy = np.array(policy_output)[0]
            # policy = tf.nn.softmax(policy_logits).numpy()[0]
            # valid_actions = game.get_valid_actions(state, prev_action)
            if prev_action is not None:
                indices = [i for i in range(game.action_space) if i != prev_action]
                try:
                    valid_actions = game.get_valid_actions(state, prev_action)
                    prob = policy[valid_actions]
                except:
                    prob = policy[indices]
                action = np.random.choice(valid_actions, p=prob / prob.sum())
            else:
                indices = list(range(game.action_space))
                prob = policy
                action = np.random.choice(indices, p=prob / prob.sum())
            if action < len(game.coupling_map):
                selected_action = game.coupling_map[action]
                swap_pairs.append(selected_action)
            else:
                for pair in game.coupling_map[action%2::2]:
                    swap_pairs.append(pair)
            state, done, _ = game.step(state, action, prev_action)
            prev_action = action
            step_count += 1
        if not done:
            depth = game.MAX_STEPS
            swap_count = game.MAX_STEPS
        else:
            depth = game.current_layer
            swap_count = len(swap_pairs)
        print(f"depth: {depth}, count: {swap_count}")
        avg_counts.append(swap_count)
        avg_depth.append(depth)
    return avg_depth, avg_counts

In [2]:
game = Game(qubits, config)
network = ResNet(action_space=game.action_space, config=config)
network = keras.models.load_model(f"checkpoints/network{qubits}_{index}_50")

In [3]:
depths = []
for _ in range(20):
    depth, count = evaluate_self_play(qubits, network, config)
    depths.append(depth)
min_depth = np.min(np.vstack(depths), axis=0)

1/1 [==============================] - 0s 75ms/step
depth: 8, count: 32
1/1 [==============================] - 0s 75ms/step
depth: 10, count: 36
1/1 [==============================] - 0s 75ms/step
depth: 40, count: 40
1/1 [==============================] - 0s 71ms/step
depth: 9, count: 40
1/1 [==============================] - 0s 75ms/step
depth: 16, count: 60
1/1 [==============================] - 0s 75ms/step
depth: 15, count: 69
1/1 [==============================] - 0s 73ms/step
depth: 11, count: 24
1/1 [==============================] - 0s 74ms/step
depth: 7, count: 33
1/1 [==============================] - 0s 70ms/step
depth: 40, count: 40
1/1 [==============================] - 0s 80ms/step
depth: 15, count: 61
1/1 [==============================] - 0s 74ms/step
depth: 14, count: 54
1/1 [==============================] - 0s 74ms/step
depth: 40, count: 40
1/1 [==============================] - 0s 74ms/step
depth: 40, count: 40
1/1 [==============================] - 0s 65ms/step
de

In [1]:
min_depth

NameError: name 'min_depth' is not defined

In [ ]:
np.mean(min_depth)

In [ ]:
np.mean(count)

In [ ]:
qubits

array([1, 5, 6, 4, 6, 7, 7, 2, 6, 3, 2, 5, 3, 7, 5, 7, 6, 3, 5, 2, 4, 9,
       2, 6, 5, 5, 3, 2, 2, 1]) -> 4.366666666666666

#7 100 3.1666  24.366666